In [ ]:
import random
import matplotlib.pyplot as plt
import seaborn
from sklearn import *
import numpy as np
import networkx as nx

np.random.seed(42)

* Fazer para geomtrico, regular, mandar email suzana
suzanadesiqueira@gmail.com

* calcular entropia (Shannon, other types too) de grafos k regulares , stochastic block model, geometrico

* revisar block stochastic model

* Fazer simulacoes e calcular e compara de forma empirica e analitica a entropia de grafos aleatorios

* https://en.wikipedia.org/wiki/Maximum-entropy_random_graph_model

* https://chih-ling-hsu.github.io/2020/05/15/Graph-Models#barabasi-albert-model

In [ ]:
def plot_graph_from_adjacency(adj_matrix):
    G = nx.Graph()  # Initialize an undirected graph
    n = len(adj_matrix)

    # Add edges to the graph
    for i in range(n):
        for j in range(n):
            if adj_matrix[i][j] == 1:
                G.add_edge(i, j)

    # Draw the graph
    nx.draw(G, with_labels=True, node_size=700, node_color="skyblue", font_size=15, font_weight='bold')
    plt.show()


# Erdos-Renyi

In [ ]:
n = 10  # número de vértices
p = 0.6  # probabilidade de uma aresta existir
m = 100  # número de simulações

def shannon_entropy(p):
    """Calcula a entropia de Shannon para uma probabilidade p."""
    if p == 0 or p == 1:  # Evitar log(0)
        return 0
    return -p * np.log2(p) - (1-p) * np.log2(1-p)

def empirical_entropy(graph):
    """Calcula a entropia empírica de um grafo."""
    total_edges_possible = n * (n - 1) / 2
    actual_edges = np.sum(graph)
    p_edge = actual_edges / total_edges_possible
    return shannon_entropy(p_edge)

# Gerar m grafos aleatórios de Erdős-Rényi e calcular a entropia empírica para cada grafo
entropies = []
for _ in range(m):
    # Gerar um grafo aleatório de Erdős-Rényi
    graph = np.random.rand(n, n) < p
    graph = np.triu(graph, 1)  # Pegar apenas a parte triangular superior para evitar duplicação
    entropies.append(empirical_entropy(graph))

# Calcular a média das entropias empíricas
mean_entropy = np.mean(entropies)
print('Mean entropy', mean_entropy)

# Calcular a entropia de Shannon analiticamente para o valor específico de p
analytical_entropy = shannon_entropy(p)
print('Ground Truth:', analytical_entropy)

Mean entropy 0.9603815621020769
Ground Truth: 0.9709505944546686


In [ ]:
#graph = np.random.rand(n,n) < .22
#plot_graph_from_adjacency(graph)

# Barabasi Albert

1. Starts with 2 connected nodes
2. Add a new node v
3. Select 1 existing nodes u such that u≠v with probability P(u)=deg(u)∑x∈Ndeg(x)
4. Create a link between u and v
5. Go back to Step 2

In [ ]:
m_0 = 3  # número inicial de vértices
m = 2  # número de arestas que cada novo vértice forma
s = 10  # número de simulações

def empirical_entropy_BA(graph):
    """Calcula a entropia empírica de um grafo Barabási-Albert."""
    total_edges_possible = n * (n - 1) / 2
    actual_edges = len(graph.edges())
    p_edge = actual_edges / total_edges_possible
    return shannon_entropy(p_edge)

# Gerar s grafos Barabási-Albert e calcular a entropia empírica para cada grafo
entropies_BA = []
for _ in range(s):
    # Gerar um grafo Barabási-Albert
    graph = nx.barabasi_albert_graph(n, m)
    entropies_BA.append(empirical_entropy_BA(graph))

# Calcular a média das entropias empíricas
mean_entropy_BA = np.mean(entropies_BA)
mean_entropy_BA


0.9389320105807949

* Barabási-Albert implica que a probabilidade de uma aresta se formar
muda a medida que o grafo cresce
* não há fórmula simples para entropia nesse caso??

# Watts-Strogatz


* https://www.kth.se/social/files/5605669af2765468be471eda/lecture%204%20%282015%29.pdf

In [ ]:
# Parâmetros fornecidos
k = 2  # cada vértice é inicialmente conectado a k vizinhos mais próximos
p = 0.4  # probabilidade de reconectar uma aresta

def empirical_entropy_WS(graph):
    """Calcula a entropia empírica de um grafo Watts-Strogatz."""
    total_edges_possible = n * (n - 1) / 2
    actual_edges = len(graph.edges())
    p_edge = actual_edges / total_edges_possible
    return shannon_entropy(p_edge)

# Gerar s grafos Watts-Strogatz e calcular a entropia empírica para cada grafo
entropies_WS = []
for _ in range(s):
    # Gerar um grafo Watts-Strogatz
    graph = nx.watts_strogatz_graph(n, k, p)
    entropies_WS.append(empirical_entropy_WS(graph))

# Calcular a média das entropias empíricas
mean_entropy_WS = np.mean(entropies_WS)
mean_entropy_WS


0.7642045065086203

* Entropia analitica? Problema similar ao barabasi albert

#Stochastic Block model

* https://epub.ub.uni-muenchen.de/69180/1/BA_GruberCornelia.pdf

#### **Concept:**
The core idea of the SBM is that nodes in a graph are grouped into a predetermined number of blocks or communities. The probability that a given edge exists between two nodes then depends solely on the communities to which the nodes belong.

#### **Parameters:**
- **Blocks/Communities:** Nodes are partitioned into \( K \) blocks or communities. Each node belongs to one and only one community.
- **Connection Probabilities:** A \( K \times K \) matrix \( P \) where \( P_{ij} \) represents the probability of an edge between a node in community \( i \) and a node in community \( j \). The diagonal elements represent the probabilities of connections within the same community, and the off-diagonal elements represent the probabilities of connections between different communities.

#### **Procedure:**
1. Divide nodes into \( K \) distinct communities.
2. For each pair of nodes, decide whether to place an edge between them based on the connection probabilities. Specifically, if nodes \( u \) and \( v \) belong to communities \( i \) and \( j \), respectively, then they are connected with probability \( P_{ij} \).

#### **Applications and Features:**
- **Community Detection:** The SBM is often used in community detection algorithms. By fitting an SBM to observed network data, one can infer the likely community membership of each node.
- **Flexibility:** The SBM can capture a wide range of network structures. It can model networks with dense connections within communities and sparse connections between communities, among other patterns.
- **Generalizations:** There are several generalizations and extensions of the SBM, like the Degree-Corrected Stochastic Block Model, which accounts for node degree heterogeneity.

#### **Limitations:**
- **Assumptions:** The SBM assumes that the only factor determining the likelihood of a connection between two nodes is the communities to which they belong. This may not capture all the complexities of real-world networks.
- **Number of Communities:** In many applications, the number of communities \( K \) must be specified in advance, which can be a limitation if this number is not known.

In [ ]:
n = 1000
z = [0, 0, 0, 0, 1, 1, 1, 1, 1, 1]  # rótulos de comunidade
p_matrix = [[0.4, 0.3], [0.3, 0.4]]  # matriz de probabilidade

def empirical_entropy_SBM(graph):
    """Calcula a entropia empírica de um grafo Stochastic Block Model."""
    total_edges_possible = n * (n - 1) / 2
    actual_edges = len(graph.edges())
    p_edge = actual_edges / total_edges_possible
    return shannon_entropy(p_edge)

# Gerar s grafos Stochastic Block Model e calcular a entropia empírica para cada grafo
entropies_SBM = []
for _ in range(s):
    # Gerar um grafo Stochastic Block Model
    graph = nx.stochastic_block_model([5, 5], p_matrix)
    entropies_SBM.append(empirical_entropy_SBM(graph))

# Calcular a média das entropias empíricas
mean_entropy_SBM = np.mean(entropies_SBM)
mean_entropy_SBM


0.0005091582926253094

Ideia Básica:
Esta abordagem considera cada combinação de blocos (ou comunidades) separadamente. Para cada par de blocos, ela calcula a entropia associada à probabilidade de uma aresta ocorrer entre os vértices desses blocos e, em seguida, toma uma média ponderada de todas essas entropias.

Passos:
1. Para cada combinação de blocos \( i \) e \( j \), determine a probabilidade \( p_{ij} \) de uma aresta entre eles usando a matriz de probabilidade \( p \).
2. Calcule a entropia de Shannon para essa probabilidade \( p_{ij} \).
3. Tome a média ponderada de todas essas entropias, ponderando pelo número de pares de vértices possíveis para cada combinação de blocos.


In [ ]:
def blockwise_entropy(i, j, community_sizes, p_matrix):
    """Calcula a entropia para arestas entre os blocos i e j."""
    p_ij = p_matrix[i][j]
    return community_sizes[i] * community_sizes[j] * shannon_entropy(p_ij)

# Calcular a entropia blockwise para cada combinação de blocos
community_sizes = [5, 5]  # Tamanho de cada comunidade
total_pairs = n * (n - 1) / 2
total_entropy = 0

for i in range(len(community_sizes)):
    for j in range(len(community_sizes)):
        total_entropy += blockwise_entropy(i, j, community_sizes, p_matrix)

# Calcular a entropia média ponderada
analytical_entropy_SBM = total_entropy / total_pairs
analytical_entropy_SBM


0.0001854095589274636

Ideia Básica: Em vez de considerar cada combinação de blocos separadamente, esta abordagem calcula diretamente a probabilidade média ponderada de uma aresta ocorrer em todo o grafo. Em seguida, ela utiliza essa probabilidade única para calcular a entropia.

Passos:

1. Calcule a probabilidade média ponderada
p_avg de uma aresta ocorrer no grafo. Isso é feito somando as probabilidades de arestas para todas as combinações de blocos, ponderadas pelo tamanho dos blocos, e dividindo pelo total de pares de vértices possíveis.

2. Calcule a entropia de Shannon para  p_avg

In [ ]:
# Calcular a probabilidade média ponderada de uma aresta ocorrer no grafo
p_avg = sum(p_matrix[i][j] * community_sizes[i] * community_sizes[j]
            for i in range(len(community_sizes))
            for j in range(len(community_sizes))) / total_pairs

# Calcular a entropia de Shannon para p_avg
analytical_entropy_SBM_direct = shannon_entropy(p_avg)
analytical_entropy_SBM_direct


0.001068112176300597